# Prepare Dataset For YOLOv7 

* `X1`,`Y1`,`X2`,`Y2` -> **Data format**

* `class_label`,`x_center`,`y_center`,`width`,`height` -> **target format**

In [8]:
import os
from math import sqrt

In [9]:
def clear_file_Content(fileContent):
    content_list = []
    # Clear the file content
    for file in fileContent:
        content_list.append(file.strip('\n'))
        
    return content_list

In [15]:
def modify_folder_data(path, folder_path):
    for filename in filter(lambda p: p.endswith("txt"), os.listdir(path)):
        filepath = os.path.join(path, filename)
        with open(filepath, mode='r+') as f:
            fileContent = f.readlines()
            cleaned_file_content = clear_file_Content(fileContent)
            new_lines = []
            for line in cleaned_file_content:
                xs = line.split(',')
                xs = [float(i) for i in xs]
                # xs[0]= x1 xs[1]= y1 xs[2]= x2 xs[3]= y2
                length1 = sqrt(xs[0]**2 + xs[1]**2)
                length2 = sqrt(xs[2]**2 + xs[3]**2)
                xs[0] = xs[0] / length1
                xs[1] = xs[1] / length1
                xs[2] = xs[2] / length2
                xs[3] = xs[3] / length2
                width = abs(xs[2] - xs[0])
                height = abs(xs[3] - xs[1])
                x_center = (xs[2]- xs[0]) / 2
                y_center = (xs[3] - xs[1]) / 2
                new_format = [0,x_center,y_center,width,height]
                new_format_string = ' '.join(str(item) for item in new_format)
                new_lines.append(new_format_string)
            with open(f"{folder_path}/{filename}", "w") as created_file:
                for line in new_lines:
                    created_file.write("%s\n" % line)
                

In [16]:
path_TrainGroundTruth = "SignatureObjectDetection\TrainGroundTruth"
path_TestGroundTruth = "SignatureObjectDetection\TestGroundTruth"

In [17]:
modify_folder_data(path_TrainGroundTruth, "SignatureObjectDetection/Data/Train/labels/")
modify_folder_data(path_TestGroundTruth, "SignatureObjectDetection/Data/Test/labels/")

# Resize All Images

In [18]:
from PIL import Image                                              
import os, sys                       

path = "SignatureObjectDetection\\TestImages\\"
dirs = os.listdir( path )                                       

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            #f, e = os.path.splitext(path+item)
            imResize = im.resize((640,640), Image.ANTIALIAS)
            imResize.save(f"SignatureObjectDetection/Data/Test/images/{item}")

resize()

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_1356\3124663361.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((640,640), Image.ANTIALIAS)


In [19]:
from PIL import Image                                              
import os, sys                       

path = "SignatureObjectDetection\\TrainImages\\"
dirs = os.listdir( path )                                       

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            #f, e = os.path.splitext(path+item)
            imResize = im.resize((640,640), Image.ANTIALIAS)
            imResize.save(f"SignatureObjectDetection/Data/Train/images/{item}")

resize()

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_1356\1231572356.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((640,640), Image.ANTIALIAS)


# Split Data into Train Validation sets

In [20]:
path = "SignatureObjectDetection\Data\Train\labels\\"
all_names = []
for filename in filter(lambda p: p.endswith("txt"), os.listdir(path)):
    all_names.append(filename.split('.')[0])

In [21]:
req = len(all_names)//5
req

132

In [22]:
import random

In [23]:
selected_names = random.sample(all_names,req )
selected_names[:10]

['463', '377', '34', '132', '192', '233', '608', '6', '597', '303']

In [24]:
import shutil

In [25]:
def train_validate_split_groundTruth(path):
    for filename in filter(lambda p: p.endswith("txt"), os.listdir(path)):
        if selected_names.__contains__(filename.split('.')[0]):
            filepath = os.path.join(path, filename) 
            shutil.move(filepath, f"SignatureObjectDetection/Data/Validation/labels/{filename}")
        
        
def train_validate_split_images(path):
    for filename in filter(lambda p: p.endswith("tif"), os.listdir(path)):
        if selected_names.__contains__(filename.split('.')[0]):
            filepath = os.path.join(path, filename) 
            shutil.move(filepath, f"SignatureObjectDetection/Data/Validation/images/{filename}")
        
        
train_validate_split_groundTruth("SignatureObjectDetection\\Data\\Train\\labels")     
train_validate_split_images("SignatureObjectDetection\\Data\\Train\\images")